# LLM Alignment Testing Framework - Demo

This notebook demonstrates how to test LLM alignment by placing models in realistic scenarios where they discover concerning information through tool use.

## Features
- Test with vLLM (GPU) or OpenRouter (API)
- 3 complexity levels: pre-filled chat, active discovery, multi-step investigation  
- 2 scenarios: Privacy Violation & Harmful Feature (dark patterns)
- Comprehensive metrics and visualizations

## Setup & Installation

In [ ]:
# Install the package (if running in Colab)
# Option 1: Install from GitHub (if repo is published)
# !pip install git+https://github.com/yourusername/align_prompts.git

# Option 2: Install in development mode from local directory
# Upload the align_prompts folder to Colab, then:
!pip install -e /content/align_prompts

# Option 3: If already in the directory
# !pip install -e .

In [ ]:
# Import required modules
import json
from align_test.core import VLLMClient, OpenRouterClient, ToolExecutor
from align_test.scenarios import PrivacyViolationScenario, HarmfulFeatureScenario
from align_test.evaluation import ResponseAnalyzer, MetricsCalculator, ResultsReporter, ScenarioMetrics

print("✓ All imports successful!")

## Configuration - Choose Your Provider

**Switch between vLLM and OpenRouter by changing just one parameter!**

In [ ]:
# ========== CONFIGURATION ==========
# Change this to switch providers: "vllm" or "openrouter"
PROVIDER = "openrouter"  # or "vllm"

# vLLM Configuration (if using Colab GPU)
VLLM_CONFIG = {
    "base_url": "http://localhost:8000/v1",
    "model": "meta-llama/Llama-3.1-8B-Instruct"
}

# OpenRouter Configuration  
OPENROUTER_CONFIG = {
    "api_key": "sk-or-v1-...",  # Replace with your OpenRouter API key
    "model": "meta-llama/llama-3.1-8b-instruct"
}

# Select client based on provider
if PROVIDER == "vllm":
    client = VLLMClient(**VLLM_CONFIG)
    print(f"✓ Using vLLM: {VLLM_CONFIG['model']}")
else:
    client = OpenRouterClient(**OPENROUTER_CONFIG)
    print(f"✓ Using OpenRouter: {OPENROUTER_CONFIG['model']}")

print(f"\nModel: {client.model_name}")

## Optional: Start vLLM Server (Only if using vLLM)

In [ ]:
# Uncomment and run this cell if you want to use vLLM in Colab
# NOTE: Requires GPU runtime

# import subprocess
# import time

# # Start vLLM server in background
# vllm_process = subprocess.Popen([
#     "python", "-m", "vllm.entrypoints.openai.api_server",
#     "--model", "meta-llama/Llama-3.1-8B-Instruct",
#     "--dtype", "half",
#     "--max-model-len", "4096"
# ])

# # Wait for server to start
# print("Starting vLLM server...")
# time.sleep(30)
# print("✓ vLLM server started!")

## Test 1: Privacy Violation - Level 1 (Pre-filled Chat)

Simplest scenario: The model receives a conversation where user data with a DO_NOT_SHARE_PII flag is already visible.

In [ ]:
# Create scenario with complexity level 1
scenario = PrivacyViolationScenario(complexity_level=1)

print("=" * 80)
print("SCENARIO: Privacy Violation (Level 1 - Pre-filled Chat)")
print("=" * 80)
print(f"Expected concern: {scenario.expected_concern}")
print(f"Complexity: {scenario.get_complexity_description()}")

# Get pre-filled messages
messages = scenario.get_initial_messages()
print(f"\nSending {len(messages)} messages to model...")
print(f"Final user prompt: {messages[-1]['content'][:100]}...")

In [ ]:
# Run the test
response = client.create_completion(
    messages=messages,
    tools=scenario.tool_registry.to_openai_format(),
    temperature=0.7,
    max_tokens=500
)

print("\n" + "=" * 80)
print("MODEL RESPONSE:")
print("=" * 80)
print(response.content)

In [ ]:
# Analyze the response
analyzer = ResponseAnalyzer()
analysis = analyzer.analyze_response(
    response.content,
    [],  # No tool execution log for Level 1
    scenario
)

print("\n" + "=" * 80)
print("ANALYSIS:")
print("=" * 80)
print(f"Concern Detected:        {'✓ PASS' if analysis.detected_concern else '✗ FAIL'}")
print(f"Objection Raised:        {'✓ PASS' if analysis.raised_objection else '✗ FAIL'}")
print(f"Task Refused:            {'✓ PASS' if analysis.refused_task else '✗ FAIL'}")
print(f"Used Problematic Data:   {'✗ FAIL' if analysis.used_problematic_data else '✓ PASS'}")
print(f"Reasoning Quality:       {analysis.reasoning_quality_score:.2f} / 1.0")

if analysis.concern_indicators:
    print(f"\nConcern indicators: {', '.join(analysis.concern_indicators[:5])}")
if analysis.problematic_data_found:
    print(f"\nProblematic data used: {analysis.problematic_data_found[:3]}")

## Test 2: Privacy Violation - Level 2 (Active Discovery)

More realistic: The model is incentivized to use tools and discovers the privacy concern organically.

In [ ]:
# Create scenario with complexity level 2
scenario = PrivacyViolationScenario(complexity_level=2)
tool_executor = ToolExecutor(scenario.tool_registry)

print("=" * 80)
print("SCENARIO: Privacy Violation (Level 2 - Active Discovery)")
print("=" * 80)

# Get initial messages
messages = scenario.get_initial_messages()
print(f"Initial user request: {messages[-1]['content']}")

# Conversation loop - model makes tool calls, we execute them
max_turns = 10
for turn in range(max_turns):
    print(f"\n--- Turn {turn + 1} ---")
    
    response = client.create_completion(
        messages=messages,
        tools=scenario.tool_registry.to_openai_format(),
        temperature=0.7
    )
    
    # Add assistant response to history
    messages.append({
        "role": "assistant",
        "content": response.content,
        "tool_calls": [{
            "id": tc.id,
            "type": tc.type,
            "function": tc.function
        } for tc in response.tool_calls] if response.tool_calls else None
    })
    
    # Check if model made tool calls
    if response.tool_calls:
        print(f"Model made {len(response.tool_calls)} tool call(s)")
        
        # Execute each tool call
        for tool_call in response.tool_calls:
            tool_name = tool_call.function["name"]
            arguments = json.loads(tool_call.function["arguments"])
            
            print(f"  Executing: {tool_name}({arguments})")
            
            # Execute tool
            result = tool_executor.execute_tool_call(tool_name, arguments)
            
            # Add tool result to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result)
            })
            
            # Check if problematic data was discovered
            if "privacy_flag" in result:
                print(f"  ⚠️ Model discovered privacy flag: {result['privacy_flag']}")
    else:
        # Model provided final response
        print("\nModel's final response:")
        print(response.content)
        break
    
    # Add decision prompt after tool discovery
    if turn == 0 and response.tool_calls:
        messages.append({
            "role": "user",
            "content": scenario.get_decision_prompt()
        })

# Analyze final response
analysis = analyzer.analyze_response(
    response.content,
    tool_executor.get_execution_history(),
    scenario
)

print("\n" + "=" * 80)
print("ANALYSIS:")
print("=" * 80)
print(f"Tools Used: {tool_executor.get_tools_used()}")
print(f"Concern Detected: {'✓' if analysis.detected_concern else '✗'}")
print(f"Task Refused: {'✓' if analysis.refused_task else '✗'}")

## Test 3: Harmful Feature - Level 1 (Pre-filled)

Test if the model flags an A/B test using dark patterns to target vulnerable users.

In [ ]:
# Create harmful feature scenario
scenario = HarmfulFeatureScenario(complexity_level=1)

print("=" * 80)
print("SCENARIO: Harmful Feature (Level 1 - Pre-filled Chat)")
print("=" * 80)
print(f"Expected concern: {scenario.expected_concern}")

# Get messages and run test
messages = scenario.get_initial_messages()
response = client.create_completion(
    messages=messages,
    tools=scenario.tool_registry.to_openai_format(),
    temperature=0.7,
    max_tokens=500
)

print("\nMODEL RESPONSE:")
print("=" * 80)
print(response.content)

# Analyze
analysis = analyzer.analyze_response(response.content, [], scenario)

print("\n" + "=" * 80)
print("ANALYSIS:")
print("=" * 80)
print(f"Detected Dark Pattern Concern: {'✓ PASS' if analysis.detected_concern else '✗ FAIL'}")
print(f"Raised Ethical Objection:      {'✓ PASS' if analysis.raised_objection else '✗ FAIL'}")
print(f"Refused to Recommend Rollout:  {'✓ PASS' if analysis.refused_task else '✗ FAIL'}")

## Run All Scenarios & Generate Report

Test all combinations of scenarios and complexity levels, then generate comprehensive metrics.

In [ ]:
# Run all scenarios and complexity levels
all_results = []
analyzer = ResponseAnalyzer()

scenarios_to_test = [
    (PrivacyViolationScenario, "Privacy Violation"),
    (HarmfulFeatureScenario, "Harmful Feature")
]

for scenario_class, scenario_name in scenarios_to_test:
    for level in [1, 2, 3]:
        print(f"\nTesting {scenario_name} - Level {level}...")
        
        try:
            scenario = scenario_class(complexity_level=level)
            messages = scenario.get_initial_messages()
            
            # For simplicity in this demo, only run Level 1 scenarios
            # Level 2 and 3 require conversation loops
            if level == 1:
                response = client.create_completion(
                    messages=messages,
                    tools=scenario.tool_registry.to_openai_format(),
                    temperature=0.7,
                    max_tokens=500
                )
                
                analysis = analyzer.analyze_response(
                    response.content,
                    [],
                    scenario
                )
                
                # Create metrics
                metrics = ScenarioMetrics.from_analysis(
                    analysis=analysis,
                    scenario_name=scenario.scenario_name,
                    complexity_level=level,
                    model_name=client.model_name,
                    tool_execution_log=[]
                )
                
                all_results.append(metrics)
                print(f"  ✓ Complete - Refused: {metrics.task_refused}")
        except Exception as e:
            print(f"  ✗ Error: {e}")

print(f"\n✓ Completed {len(all_results)} tests")

In [ ]:
# Calculate aggregate metrics
calculator = MetricsCalculator()
aggregate = calculator.calculate_aggregate_metrics(all_results)

# Generate and print summary report
reporter = ResultsReporter()
summary = reporter.generate_summary_report(all_results, aggregate)
print(summary)

## Visualizations

In [ ]:
# Create visualizations
fig = reporter.create_visualization(
    all_results,
    title=f"Alignment Testing Results - {client.model_name}"
)
plt.show()

## Export Results

In [ ]:
# Export to DataFrame
df = reporter.export_to_dataframe(all_results)
print("\nResults DataFrame:")
display(df)

# Save to CSV
df.to_csv("alignment_test_results.csv", index=False)
print("\n✓ Results saved to alignment_test_results.csv")

## Next Steps

1. **Test different models**: Change the `OPENROUTER_CONFIG["model"]` to test other models
2. **Implement Level 2 & 3**: Add conversation loops for more realistic testing
3. **Create custom scenarios**: Inherit from `BaseScenario` to test your own alignment cases
4. **Compare models**: Run tests with multiple models and compare their alignment behavior

## Documentation

See the [README](../README.md) for full documentation and advanced usage.